In [1]:
import os
import glob
import xarray as xr 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def select_latlon(ds):
    ds = ds.drop_vars('number').drop_vars('pressure_level').squeeze('pressure_level')
    ds_coarse = ds.coarsen(latitude=5, longitude=5, boundary="trim").mean()
    return ds_coarse.sel(latitude = slice(90, 20), longitude = slice(-90, 90))

dy = xr.open_mfdataset('/gws/nopw/j04/aopp/tildah/zg_data/ERA5-meanZ500/ERA5_mean_Z500_*.nc', 
                       combine='by_coords',  
                       preprocess=select_latlon)

dy = dy.rename({
    "longitude": "lon",
    "latitude": "lat",
    "valid_time": "time",
    "z": "zg500"
})

In [23]:
zg500raw = dy.isel(time=dy['time'].dt.month.isin([6, 7, 8])).sel(time=slice('1940-06-01', '2014-08-31'))['zg500']
#print(zg500raw)
climatology = zg500raw.sel(time=slice("1981-06-01", "2010-08-31")).groupby("time.dayofyear").mean(dim="time")
zg500anom = zg500raw.groupby("time.dayofyear") - climatology
#print(zg500anom)

hw_input = xr.open_dataset('ERA5_heatwave_in.nc')
hw_days = hw_input['heatwave_in_europe'].drop_vars('valid_time').squeeze("valid_time", drop=True)
hw_days = hw_days.isel(time=hw_days['time'].dt.month.isin([6, 7, 8])).sel(time=slice("1940-06-01", "2014-08-31"))
print(hw_days)

zg500anom_hw = zg500anom.where(hw_days, drop=True)
print(zg500anom_hw)

zg500arr = zg500anom_hw.stack(point=["lat", "lon"])
print(zg500arr)

np.save('ERA5_data_train.npy', zg500arr)
zg500anom_hw.coords.to_dataset().to_netcdf("ERA5_coords.nc")


<xarray.DataArray 'heatwave_in_europe' (time: 6900)> Size: 7kB
[6900 values with dtype=bool]
Coordinates:
  * time       (time) datetime64[ns] 55kB 1940-06-01 1940-06-02 ... 2014-08-31
    number     int64 8B ...
    dayofyear  (time) int64 55kB ...
    quantile   float64 8B ...
<xarray.DataArray 'zg500' (time: 698, lat: 56, lon: 144)> Size: 23MB
dask.array<where, shape=(698, 56, 144), dtype=float32, chunksize=(1, 36, 72), chunktype=numpy.ndarray>
Coordinates:
  * lat       (lat) float64 448B 89.5 88.25 87.0 85.75 ... 24.5 23.25 22.0 20.75
  * lon       (lon) float64 1kB -89.5 -88.25 -87.0 -85.75 ... 86.75 88.0 89.25
  * time      (time) datetime64[ns] 6kB 1940-06-25 1940-06-26 ... 2014-08-15
    number    int64 8B 0
    quantile  float64 8B 0.9
<xarray.DataArray 'zg500' (time: 698, point: 8064)> Size: 23MB
dask.array<reshape, shape=(698, 8064), dtype=float32, chunksize=(1, 2592), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) datetime64[ns] 6kB 1940-06-25 1940-06-26 ... 20

In [4]:
zg500raw = dy.isel(time=dy['time'].dt.month.isin([6, 7, 8])).sel(time=slice('1940-06-01', '2014-08-31'))['zg500']
#print(zg500raw)
climatology = zg500raw.sel(time=slice("1981-06-01", "2010-08-31")).groupby("time.dayofyear").mean(dim="time")
zg500anom = zg500raw.groupby("time.dayofyear") - climatology
#print(zg500anom)

zg500arr = zg500anom.stack(point=["lat", "lon"])
print(zg500arr)

np.save('ERA5_data_train_allJJA.npy', zg500arr)
zg500anom.coords.to_dataset().to_netcdf("ERA5_coords_allJJA.nc")

<xarray.DataArray 'zg500' (time: 6900, point: 8064)> Size: 223MB
dask.array<reshape, shape=(6900, 8064), dtype=float32, chunksize=(1, 2592), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 55kB 1940-06-01 1940-06-02 ... 2014-08-31
    dayofyear  (time) int64 55kB 153 154 155 156 157 158 ... 239 240 241 242 243
  * point      (point) object 65kB MultiIndex
  * lat        (point) float64 65kB 89.5 89.5 89.5 89.5 ... 20.75 20.75 20.75
  * lon        (point) float64 65kB -89.5 -88.25 -87.0 ... 86.75 88.0 89.25
